<a href="https://colab.research.google.com/github/cottonbhimireddy1/-transaction-data-pipeline/blob/main/transaction_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd



In [59]:
path = ("/content/transaction_data.csv")
tx = pd.read_csv(path)

tx.shape


(2595732, 12)

In [60]:
tx.columns.tolist()


['household_key',
 'BASKET_ID',
 'DAY',
 'PRODUCT_ID',
 'QUANTITY',
 'SALES_VALUE',
 'STORE_ID',
 'RETAIL_DISC',
 'TRANS_TIME',
 'WEEK_NO',
 'COUPON_DISC',
 'COUPON_MATCH_DISC']

In [61]:
tx.head()


,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [62]:
tx.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595732 entries, 0 to 2595731
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   household_key      int64  
 1   BASKET_ID          int64  
 2   DAY                int64  
 3   PRODUCT_ID         int64  
 4   QUANTITY           int64  
 5   SALES_VALUE        float64
 6   STORE_ID           int64  
 7   RETAIL_DISC        float64
 8   TRANS_TIME         int64  
 9   WEEK_NO            int64  
 10  COUPON_DISC        float64
 11  COUPON_MATCH_DISC  float64
dtypes: float64(4), int64(8)
memory usage: 237.6 MB


In [63]:
tx["TRANS_TIME"] = pd.to_datetime(tx["TRANS_TIME"], errors="coerce")

tx["TRANS_TIME"].min(), tx["TRANS_TIME"].max(), tx["TRANS_TIME"].isna().sum()


(Timestamp('1970-01-01 00:00:00'),
 Timestamp('1970-01-01 00:00:00.000002359'),
 np.int64(0))

In [64]:
tx["household_key"].nunique(), len(tx)


(2500, 2595732)

In [65]:
# Convert sales_value to numeric safely (in case it's read as string)
tx["SALES_VALUE"] = pd.to_numeric(tx["SALES_VALUE"], errors="coerce")

tx["SALES_VALUE"].isna().sum(), (tx["SALES_VALUE"] == 0).sum(), (tx["SALES_VALUE"] < 0).sum()


(np.int64(0), np.int64(18850), np.int64(0))

In [66]:
tx["SALES_VALUE"].describe(percentiles=[0.5, 0.9, 0.95, 0.99])


,SALES_VALUE
count,2.595732e+06
mean,3.104120e+00
std,4.182274e+00
min,0.000000e+00
50%,2.000000e+00
90%,5.820000e+00
95%,7.990000e+00
99%,2.000000e+01
max,8.400000e+02


In [67]:
# Candidate date columns (dunnhumby often uses DAY)
[c for c in tx.columns if "day" in c.lower() or "date" in c.lower()]


['DAY']

In [68]:
DATE_COL = "DAY"  # <-- replace if different

# Ensure numeric day (some files read it as string)
tx[DATE_COL] = pd.to_numeric(tx[DATE_COL], errors="coerce")

# Create a month index from day (4-week months are fine for this dataset)
tx["month_index"] = (tx[DATE_COL] // 30)  # simple month bucket
tx[["month_index", DATE_COL]].head()


,month_index,DAY
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [69]:
# Ensure sales_value numeric
tx["SALES_VALUE"] = pd.to_numeric(tx["SALES_VALUE"], errors="coerce")

cust_month = (
    tx.groupby(["household_key", "month_index"], as_index=False)
      .agg(
          total_spend=("SALES_VALUE", "sum"),
          txns=("BASKET_ID", "nunique"),
          line_items=("PRODUCT_ID", "count"),
          avg_line_value=("SALES_VALUE", "mean"),
          median_line_value=("SALES_VALUE", "median"),
      )
)

cust_month.head()


,household_key,month_index,total_spend,txns,line_items,avg_line_value,median_line_value
0,1,1,78.66,1,30,2.622000,2.500
1,1,2,68.00,2,24,2.833333,2.420
2,1,3,167.19,4,59,2.833729,2.500
3,1,4,271.10,3,104,2.606731,2.425
4,1,5,237.93,4,93,2.558387,2.180


In [70]:
raw_spend = tx["SALES_VALUE"].sum()
agg_spend = cust_month["total_spend"].sum()

raw_spend, agg_spend, raw_spend - agg_spend


(np.float64(8057463.079999994),
 np.float64(8057463.08),
 np.float64(-6.51925802230835e-09))

In [71]:
cust_month["total_spend"].describe(percentiles=[0.5, 0.9, 0.95, 0.99])


,total_spend
count,45439.000000
mean,177.324833
std,189.177173
min,0.000000
50%,116.060000
90%,416.792000
95%,552.721000
99%,870.917800
max,2803.550000


In [72]:
# Identify extreme spend months (top 1%)
p99 = cust_month["total_spend"].quantile(0.99)

cust_month[cust_month["total_spend"] >= p99].head(), p99


(     household_key  month_index  total_spend  txns  line_items  avg_line_value  median_line_value
 182             13            4       902.51    16         166        5.436807              2.585
 187             13            9       890.38    17         171        5.206901              2.780
 197             13           19       939.75    21         175        5.370000              2.990
 319             19           15      1006.78    43         374        2.691925              2.000
 321             19           17       957.87    35         399        2.400677              1.890,
 np.float64(870.9178000000005))

In [73]:
CAP_VALUE = cust_month["total_spend"].quantile(0.99)

cust_month["total_spend_capped"] = cust_month["total_spend"].clip(upper=CAP_VALUE)

cust_month[["total_spend", "total_spend_capped"]].describe(percentiles=[0.9, 0.95, 0.99])



,total_spend,total_spend_capped
count,45439.000000,45439.000000
mean,177.324833,175.182424
std,189.177173,177.913071
min,0.000000,0.000000
50%,116.060000,116.060000
90%,416.792000,416.792000
95%,552.721000,552.721000
99%,870.917800,870.873036
max,2803.550000,870.917800


In [74]:
cust_month[[
    "total_spend",
    "total_spend_capped",
    "txns",
    "line_items",
    "avg_line_value",
    "median_line_value"
]].isna().sum(), (cust_month["total_spend"] < 0).sum(), (cust_month["txns"] <= 0).sum()


(total_spend           0
 total_spend_capped    0
 txns                  0
 line_items            0
 avg_line_value        0
 median_line_value     0
 dtype: int64,
 np.int64(0),
 np.int64(0))

In [75]:
missing = cust_month[[
    "total_spend","total_spend_capped","txns","line_items","avg_line_value","median_line_value"
]].isna().sum()

neg_spend = (cust_month["total_spend"] < 0).sum()
zero_txn_months = (cust_month["txns"] <= 0).sum()

missing, neg_spend, zero_txn_months


(total_spend           0
 total_spend_capped    0
 txns                  0
 line_items            0
 avg_line_value        0
 median_line_value     0
 dtype: int64,
 np.int64(0),
 np.int64(0))

In [76]:
cust_month.groupby("month_index")[[
    "total_spend_capped",
    "txns",
    "avg_line_value"
]].agg(["mean", "std"]).head()


total_spend_capped                  txns           avg_line_value          
                          mean         std      mean       std           mean       std
month_index                                                                            
0                   104.844079  128.745358  3.567134  4.090468       3.555488  9.150074
1                   123.351523  138.088195  4.421384  4.778472       3.335134  2.301795
2                   138.388604  148.602512  5.149642  5.921227       3.671980  5.962922
3                   145.306250  153.548804  5.519481  6.607181       3.388246  2.985954
4                   167.958222  169.826460  6.510521  7.604536       3.320402  2.630580

In [77]:
cust_month[["txns", "total_spend_capped"]].corr(method="spearman")


,txns,total_spend_capped
txns,1.000000,0.749857
total_spend_capped,0.749857,1.000000


In [78]:
import os

output_dir = "/mnt/data"
os.makedirs(output_dir, exist_ok=True)

output_path = f"{output_dir}/customer_month_pipeline_output.csv"
cust_month.to_csv(output_path, index=False)

output_path


'/mnt/data/customer_month_pipeline_output.csv'

In [79]:
{
    "rows": cust_month.shape[0],
    "columns": cust_month.shape[1],
    "total_spend_preserved": cust_month["total_spend"].sum(),
    "missing_values": cust_month.isna().sum().sum(),
    "negative_spend_rows": (cust_month["total_spend"] < 0).sum(),
    "zero_txn_months": (cust_month["txns"] <= 0).sum(),
    "cap_strategy": "99th percentile capping applied to total_spend"
}


{'rows': 45439,
 'columns': 8,
 'total_spend_preserved': np.float64(8057463.08),
 'missing_values': np.int64(0),
 'negative_spend_rows': np.int64(0),
 'zero_txn_months': np.int64(0),
 'cap_strategy': '99th percentile capping applied to total_spend'}